## Introduction
This notebook provides an interactive environment for training a predictive model to estimate patient length of stay (LOS). This workflow is particularly suited for healthcare professionals and researchers interested in ICU resource planning and patient flow management.

Users are only requested to change the code block indicated with 

In [ ]:
### ‼️User Action Required

All the other blocks should work without interference.
Warnings and notices are preceded by ⚠️

### Package Installation
This code checks for the presence of required R packages and installs them if they are not already available.
These packages are essential for data preprocessing, model training, ensemble learning, and performance evaluation:
- `caret`: Core package for building and tuning predictive models.
- `caretEnsemble`: Allows combining multiple caret models into an ensemble for improved accuracy.
- `tidyverse`: Collection of packages for data manipulation, visualization, and general workflow.
- `MLmetrics`: Provides machine learning evaluation metrics (e.g., MAE, RMSE).
- `ranger`: Fast implementation of Random Forests, useful for training tree-based models efficiently.

⚠️ You only need to run this block once per session or when setting up a new environment.


In [1]:
if (!require("caret")) {install.packages("caret", dependencies = TRUE) ; library(caret)}
if (!require("caretEnsemble")) {install.packages("caretEnsemble", dependencies = TRUE) ; library(caretEnsemble)}
if (!require("tidyverse")) {install.packages("tidyverse") ; library(tidyverse)}
if (!require("MLmetrics")) {install.packages("MLmetrics") ; library(MLmetrics)}
if (!require("ranger")) {install.packages("ranger"); library(ranger) }
if(!require(DescTools)) {install.packages("DescTools"); library(DescTools) }
if(!require(mice)) {install.packages("mice"); library(mice) }

Carregando pacotes exigidos: caret

Carregando pacotes exigidos: ggplot2

Warning message:
"pacote 'ggplot2' foi compilado no R versão 4.4.2"
Carregando pacotes exigidos: lattice

Carregando pacotes exigidos: caretEnsemble

Carregando pacotes exigidos: tidyverse

── Attaching core tidyverse packages ──────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ lubridate 1.9.3     ✔ tibble    3.2.1
✔ purrr     1.0.2     ✔ tidyr     1.3.1
── Conflicts ────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
✖ purrr::lift()   masks caret::lift()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Carregando pacotes exigidos: MLmetrics


Anexando pacote: 'MLmetrics'


Os seguintes objetos são mascarados por 'pack

pacote 'rootSolve' desempacotado com sucesso e somas MD5 verificadas
pacote 'lmom' desempacotado com sucesso e somas MD5 verificadas
pacote 'expm' desempacotado com sucesso e somas MD5 verificadas
pacote 'Exact' desempacotado com sucesso e somas MD5 verificadas
pacote 'gld' desempacotado com sucesso e somas MD5 verificadas
pacote 'DescTools' desempacotado com sucesso e somas MD5 verificadas

Os pacotes binários baixados estão em
	C:\Users\joana\AppData\Local\Temp\RtmpigiNqe\downloaded_packages


Warning message:
"pacote 'DescTools' foi compilado no R versão 4.4.3"

Anexando pacote: 'DescTools'


Os seguintes objetos são mascarados por 'package:MLmetrics':

    AUC, Gini, MAE, MAPE, MSE, RMSE


Os seguintes objetos são mascarados por 'package:caret':

    MAE, RMSE


Carregando pacotes exigidos: mice

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
"não há nenhum pacote chamado 'mice'"
Instalando pacote em 'C:/Users/joana/AppData/Local/R/win-library/4.4'
(como 'lib' não foi especificado)

instalando as dependências 'ucminf', 'ordinal', 'pan', 'jomo', 'glmnet', 'mitml' também





  Existe uma versão binária disponível, mas a versão de código-fonte é
  posterior:
       binary source needs_compilation
glmnet  4.1-8  4.1-9              TRUE

pacote 'ucminf' desempacotado com sucesso e somas MD5 verificadas
pacote 'ordinal' desempacotado com sucesso e somas MD5 verificadas
pacote 'pan' desempacotado com sucesso e somas MD5 verificadas
pacote 'jomo' desempacotado com sucesso e somas MD5 verificadas
pacote 'mitml' desempacotado com sucesso e somas MD5 verificadas
pacote 'mice' desempacotado com sucesso e somas MD5 verificadas

Os pacotes binários baixados estão em
	C:\Users\joana\AppData\Local\Temp\RtmpigiNqe\downloaded_packages


instalando o pacote de código-fonte 'glmnet'


Warning message:
"pacote 'mice' foi compilado no R versão 4.4.3"

Anexando pacote: 'mice'


O seguinte objeto é mascarado por 'package:stats':

    filter


Os seguintes objetos são mascarados por 'package:base':

    cbind, rbind




### Load and Validate User Dataset

1. Please change the `data_path` the **path to the data** you want to train the model on. It can be in `.csv` or `.RData` format. If it's an `R.Data` file, please include the object name in the `object_name` variable.

2. If you want to include your **own predictors**, please change the `predictors` variable to include your a dataframe with one column stating the names of your predictors.

If not, the the list of predictor variables used during the original model training is automatically loaded.

⚠️ If not using your own predictors, make sure your dataset includes all required predictors listed in predictors.csv, as well as the target variable UnitLengthStay_trunc.

In [12]:
### ‼️User Action Required

#data_path = "YOUR_PATH"
data_path = "C:\\Users\\joana\\Documentos\\SLOS\\SLOS retraining\\SampledData.RData"
object_name = "sampled_data"
predictors = "C:\\Users\\joana\\Documentos\\SLOS\\SLOS retraining\\predictors.csv"

In [13]:
if (grepl("\\.csv$", data_path, ignore.case = TRUE)) {
  user_data <- read.csv(data_path)
} else if (grepl("\\.RData$", data_path, ignore.case = TRUE)) {
  load(data_path)
  if (!exists(object_name)) {
    stop(paste("The .RData file does not contain an object named", object_name))
  }
  user_data <- get(object_name)
} else {
  stop("Unsupported file type. Please upload a .csv or .RData file.")
}

predictors <- read.csv(predictors)
predictors <- predictors[,2]  

if (!all(predictors %in% names(user_data))) {
  stop("Some required predictors are missing in your dataset.")
}

user_data <- user_data %>%
  select(all_of(predictors), UnitLengthStay_trunc)

In [65]:
unique(testing$AdmissionTypeName)

[1] Medical           Scheduled surgery Emergency surgery
Levels: Emergency surgery Medical Scheduled surgery Surgical

### Data Pre-processing
We remove zero and near-zero variance features, correlated predictors (for numeric and categorical features) and we impute missing data via the MICE algortihm

In [67]:
set.seed(998)
inTraining <- createDataPartition(user_data$UnitLengthStay,
                                  p = .8, list = FALSE)
training <- user_data[ inTraining,]
training_dummy <- training
testing  <- user_data[-inTraining,]
testing_dummy <- testing

#Identifying and Removing Zero and Near Zero variance features
nzv = nearZeroVar(training, saveMetrics = T, freqCut = 100/2)
nzv["Variaveis"] = row.names(nzv)
descritiva_nzv = nzv%>%
  filter(nzv==T)%>%
  select(Variaveis,freqRatio,percentUnique)
retirados_nzv = descritiva_nzv$Variaveis

training = training %>%
  select(.,-retirados_nzv)
testing = testing %>%
  select(.,-retirados_nzv)

# Identifying and Removing Correlated Predictors (for numeric features)
training_pre_numeric = training %>%
  select_if(., is.numeric)
training_pre_numeric$UnitLengthStay = NULL
descrCor <-  cor(training_pre_numeric, 
                 use="pairwise.complete.obs")

highlyCorDescr <- findCorrelation(descrCor, cutoff = .75)
retirados_cor = colnames(training_pre_numeric[,highlyCorDescr])
training_pre_numeric = 
  training_pre_numeric[,-highlyCorDescr]

testing_pre_numeric = testing %>%
  select_if(., is.numeric)
testing_pre_numeric$UnitLengthStay = NULL
testing_pre_numeric = 
  testing_pre_numeric[,-highlyCorDescr]


# Identifying and Removing Correlated Predictors (for categorical features)
training_pre_factor = training %>%
  select_if(., is.factor)
cramer_tab = PairApply(training_pre_factor,
                       CramerV, symmetric = TRUE)
cramer_tab[which(is.na(cramer_tab[,])==T)] = 0

highlyCorCateg <- findCorrelation(cramer_tab, cutoff = 0.5)
retirados_categ = colnames(training_pre_factor[,highlyCorCateg])
training_pre_factor = training_pre_factor %>%
  select(.,-retirados_categ)

testing_pre_factor = testing %>%
  select_if(., is.factor)
testing_pre_factor = testing_pre_factor %>%
  select(.,-retirados_categ)

training = cbind(training_pre_numeric,training_pre_factor, training$UnitLengthStay)
training$UnitLengthStay = training$`training$UnitLengthStay`
training$`training$UnitLengthStay` = NULL

testing = cbind(testing_pre_numeric,testing_pre_factor, testing$UnitLengthStay)
testing$UnitLengthStay = testing$`testing$UnitLengthStay`
testing$`testing$UnitLengthStay` = NULL


#MICE Imputation
training_imp = training
testing_imp = testing

  #training
set.seed(100)
predictormatrix = quickpred(training_imp,
                          include = c("UnitLengthStay"),
                          exclude = NULL,
                          mincor = 0.3)
imp_gen = mice(data = training_imp,
               predictorMatrix = predictormatrix,
               m=1,
               maxit = 5,
               diagnostics=TRUE)

imp_data = mice::complete(imp_gen,1)
training_imp = imp_data
summary(training_imp)
training_imp$UnitLengthStay_trunc <- training_dummy$UnitLengthStay_trunc
training <- training_imp


  #testing
set.seed(100)
predictormatrix = quickpred(testing_imp,
                            include = c("UnitLengthStay"),
                            exclude = NULL,
                            mincor = 0.3)
imp_gen_test = mice(data = testing_imp,
               predictorMatrix = predictormatrix,
               m=1,
               maxit = 5,
               diagnostics=TRUE)
imp_data_test = mice::complete(imp_gen_test,1)
testing_imp = imp_data_test
summary(testing_imp)
testing_imp$UnitLengthStay_trunc <- testing_dummy$UnitLengthStay_trunc
testing <- testing_imp


 iter imp variable
  1   1
  2   1
  3   1
  4   1
  5   1


 IsMechanicalVentilation IsVasopressors
 0:776                   0:764         
 1: 24                   1: 36         
                                       
                                       
                                       
                                       
                       AdmissionSourceName         AdmissionTypeName
 Cardiovascular intervention room: 18      Emergency surgery: 44    
 Emergency room                  :553      Medical          :676    
 Operating room                  : 81      Scheduled surgery: 79    
 Other                           : 37      Surgical         :  1    
 Other unit at your hospital     : 76                               
 Ward/Floor                      : 35                               
 IsNonInvasiveVentilation IsRespiratoryFailure IsDementia    ChfNyha    Gender 
 0:748                    0:764                0:734      Class23: 40   F:453  
 1: 52                    1: 36                1: 66      Class4 : 12   M:347


 iter imp variable
  1   1
  2   1
  3   1
  4   1
  5   1


 IsMechanicalVentilation IsVasopressors
 0:192                   0:192         
 1:  8                   1:  8         
                                       
                                       
                                       
                                       
                       AdmissionSourceName         AdmissionTypeName
 Cardiovascular intervention room:  5      Emergency surgery:  6    
 Emergency room                  :143      Medical          :169    
 Operating room                  : 17      Scheduled surgery: 25    
 Other                           :  6      Surgical         :  0    
 Other unit at your hospital     : 23                               
 Ward/Floor                      :  6                               
 IsNonInvasiveVentilation IsRespiratoryFailure IsDementia    ChfNyha    Gender 
 0:186                    0:192                0:183      Class23: 13   F: 94  
 1: 14                    1:  8                1: 17      Class4 :  3   M:106

### Model Traning
This section covers the full training pipeline, from splitting the data to building an ensemble model using caretStack.

**Steps**:
1. **Train-Test Split**

- A reproducible 80/20 split is created using createDataPartition().

2. **Cross-Validation Setup**

- `trainControl()` defines a 5-fold cross-validation strategy with progress output (verboseIter) and final model predictions retained (savePredictions = "final")

3. **Model Training with caretList**

- Two base models are trained using caretList(): Linear regression (lm) and Random Forest (ranger) with a tuning grid for mtry, splitrule, and min.node.size. These models are stored in model_list and saved as "user_model_list.RData".

4. **Model Stacking with caretStack**

- A stacked ensemble model is built from the base learners using caretStack(). A secondary Random Forest model (with its own tuning grid) is used to combine the predictions. The final stacked model is saved as "user_trained_SLOS_model.RData".

This ensures both the individual models and the stacked model can be reused or deployed later.



In [46]:
fitControl <- trainControl(
  method = "cv", 
  number = 5, 
  verboseIter = TRUE, 
  returnData = FALSE,
  trim = TRUE,
  savePredictions = "final"
)

In [47]:
model_list <- caretList(
  x = training[, -ncol(training)],
  y = training$UnitLengthStay_trunc,
  trControl = fitControl,
  metric = "RMSE",
  tuneList = list(
    lm = caretModelSpec(method = "lm"),
    rf = caretModelSpec(method = "ranger", tuneGrid = data.frame(
      .mtry = c(5:10),
      .splitrule = "variance",
      .min.node.size = 5
    ))
  )
)

save(model_list, file = "user_model_list.RData")

+ Fold1: intercept=TRUE 
- Fold1: intercept=TRUE 
+ Fold2: intercept=TRUE 
predictions failed for Fold2: intercept=TRUE Error in model.frame.default(Terms, newdata, na.action = na.action, xlev = object$xlevels) : 
  fator 'AdmissionTypeName' tem novos níveis Surgical
 


Warning message:
"predictions failed for Fold2: intercept=TRUE Error in model.frame.default(Terms, newdata, na.action = na.action, xlev = object$xlevels) : 
  fator 'AdmissionTypeName' tem novos níveis Surgical
"


- Fold2: intercept=TRUE 
+ Fold3: intercept=TRUE 
- Fold3: intercept=TRUE 
+ Fold4: intercept=TRUE 
- Fold4: intercept=TRUE 
+ Fold5: intercept=TRUE 
- Fold5: intercept=TRUE 


Warning message in nominalTrainWorkflow(x = x, y = y, wts = weights, info = trainInfo, :
"There were missing values in resampled performance measures."


Aggregating results
Fitting final model on full training set
+ Fold1: mtry= 5, splitrule=variance, min.node.size=5 
- Fold1: mtry= 5, splitrule=variance, min.node.size=5 
+ Fold1: mtry= 6, splitrule=variance, min.node.size=5 
- Fold1: mtry= 6, splitrule=variance, min.node.size=5 
+ Fold1: mtry= 7, splitrule=variance, min.node.size=5 
- Fold1: mtry= 7, splitrule=variance, min.node.size=5 
+ Fold1: mtry= 8, splitrule=variance, min.node.size=5 
- Fold1: mtry= 8, splitrule=variance, min.node.size=5 
+ Fold1: mtry= 9, splitrule=variance, min.node.size=5 
- Fold1: mtry= 9, splitrule=variance, min.node.size=5 
+ Fold1: mtry=10, splitrule=variance, min.node.size=5 
- Fold1: mtry=10, splitrule=variance, min.node.size=5 
+ Fold2: mtry= 5, splitrule=variance, min.node.size=5 
- Fold2: mtry= 5, splitrule=variance, min.node.size=5 
+ Fold2: mtry= 6, splitrule=variance, min.node.size=5 
- Fold2: mtry= 6, splitrule=variance, min.node.size=5 
+ Fold2: mtry= 7, splitrule=variance, min.node.size=5 
- Fo

In [48]:
rfGrid <- expand.grid(
  mtry = 2,
  min.node.size = c(5,10,15,20),
  splitrule = c("variance", "extratrees", "maxstat")
)

stacked_model <- caretStack(
  model_list,
  trControl = fitControl,
  metric = "RMSE",
  method = "ranger",
  tuneGrid = rfGrid
)

save(stacked_model, file = "user_trained_SLOS_model.RData")

+ Fold1: mtry=2, min.node.size= 5, splitrule=variance 
model fit failed for Fold1: mtry=2, min.node.size= 5, splitrule=variance Error : Missing data in columns: lm.
 


Warning message:
"model fit failed for Fold1: mtry=2, min.node.size= 5, splitrule=variance Error : Missing data in columns: lm.
"


- Fold1: mtry=2, min.node.size= 5, splitrule=variance 
+ Fold1: mtry=2, min.node.size=10, splitrule=variance 
model fit failed for Fold1: mtry=2, min.node.size=10, splitrule=variance Error : Missing data in columns: lm.
 


Warning message:
"model fit failed for Fold1: mtry=2, min.node.size=10, splitrule=variance Error : Missing data in columns: lm.
"


- Fold1: mtry=2, min.node.size=10, splitrule=variance 
+ Fold1: mtry=2, min.node.size=15, splitrule=variance 
model fit failed for Fold1: mtry=2, min.node.size=15, splitrule=variance Error : Missing data in columns: lm.
 


...

Warning message:
"model fit failed for Fold5: mtry=2, min.node.size=20, splitrule=maxstat Error : Missing data in columns: lm.
"


- Fold5: mtry=2, min.node.size=20, splitrule=maxstat 


Warning message in nominalTrainWorkflow(x = x, y = y, wts = weights, info = trainInfo, :
"There were missing values in resampled performance measures."


Aggregating results
Something is wrong; all the RMSE metric values are missing:
      RMSE        Rsquared        MAE     
 Min.   : NA   Min.   : NA   Min.   : NA  
 1st Qu.: NA   1st Qu.: NA   1st Qu.: NA  
 Median : NA   Median : NA   Median : NA  
 Mean   :NaN   Mean   :NaN   Mean   :NaN  
 3rd Qu.: NA   3rd Qu.: NA   3rd Qu.: NA  
 Max.   : NA   Max.   : NA   Max.   : NA  
 NA's   :12    NA's   :12    NA's   :12   


ERROR: Error: Stopping


In [54]:
train(
  x = training[, !(names(training) %in% c("UnitLengthStay_trunc"))],
  y = training$UnitLengthStay_trunc,
  method = "lm"
)


Warning message:
"predictions failed for Resample01: intercept=TRUE Error in model.frame.default(Terms, newdata, na.action = na.action, xlev = object$xlevels) : 
  fator 'AdmissionTypeName' tem novos níveis Surgical
"
Warning message:
"predictions failed for Resample02: intercept=TRUE Error in model.frame.default(Terms, newdata, na.action = na.action, xlev = object$xlevels) : 
  fator 'AdmissionTypeName' tem novos níveis Surgical
"
Warning message:
"predictions failed for Resample05: intercept=TRUE Error in model.frame.default(Terms, newdata, na.action = na.action, xlev = object$xlevels) : 
  fator 'AdmissionTypeName' tem novos níveis Surgical
"
Warning message:
"predictions failed for Resample10: intercept=TRUE Error in model.frame.default(Terms, newdata, na.action = na.action, xlev = object$xlevels) : 
  fator 'AdmissionTypeName' tem novos níveis Surgical
"
Warning message:
"predictions failed for Resample11: intercept=TRUE Error in model.frame.default(Terms, newdata, na.action = na.

Linear Regression 

800 samples
 18 predictor

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 800, 800, 800, 800, 800, 800, ... 
Resampling results:

  RMSE          Rsquared  MAE         
  5.126018e-15  1         3.453253e-15

Tuning parameter 'intercept' was held constant at a value of TRUE

### Model Prediction and Evaluation
This section handles making predictions with the trained stacked model and evaluating its performance using key metrics.

1. the predict(stacked_model, newdata = testing) call generates **predictions** for the test set using the stacked model.

2. We **evaluate** the trained model performance on three metrics:

- Root Mean Squared Error (RMSE): Measures the average magnitude of the prediction errors.

- Mean Absolute Error (MAE): Measures the average of the absolute errors, giving a sense of how far off the predictions are.

- R-squared (R2): Indicates the proportion of the variance in the dependent variable that is predictable from the independent variables.

These metrics are computed using the functions available in the MLMetrics package.

In [49]:
predictions <- predict(stacked_model, newdata = testing)

In [50]:
testing$UnitLengthStay_trunc

[1]  2.6215278  7.1354167  1.0465278  2.7708333  3.8847222 21.0000000
  [7]  1.1145833  5.3520833  3.0763889  2.4743056  0.9312500  2.7847222
 [13]  1.1875000  0.6187500 21.0000000  0.7944444 20.7562500  8.5361111
 [19]  9.8576389  2.9715278 17.4708333  8.8631944  1.7534722  6.6388889
 [25]  1.8972222  2.8868056  2.8930556  1.9659722 10.7131944  3.0097222
 [31]  0.7784722  3.8111111  1.9388889  4.0590278  3.0868056  8.5395833
 [37]  3.2701389  1.7708333  7.7833333  8.7305556  5.0527778  8.0104167
 [43]  2.0152778  1.7576389  0.9097222  1.4645833  9.0972222  2.7256944
 [49]  4.9375000  2.2340278  1.5972222  6.9875000  2.1298611  1.5777778
 [55]  2.3326389  6.9069444  1.1861111  2.7375000  1.6145833  2.3708333
 [61]  4.0125000  1.9812500  2.7812500  2.4090278  0.9576389  3.6819444
 [67]  2.0479167  1.9736111  4.2708333  6.0756944  1.9236111  1.9513889
 [73]  3.3020833  0.8111111  3.6333333  2.7291667  3.0069444  0.8972222
 [79]  2.4541667  3.7750000  1.7201389  0.8805556  1.3270833  2.8333333
 [85]  2.9381944 12.8118056  0.9277778  1.7833333  9.6055556  2.9375000
 [91]  3.9555556  0.3791667  7.0000000  4.4020833  4.8673611  1.2590278
 [97] 10.6673611  6.3236111  1.8354167  1.6527778  6.9319444  2.8444444
[103]  1.3201389  0.7041667 10.0395833  3.6041667  2.1451389  1.3944444
[109]  1.9020833 21.0000000  7.5847222  2.7222222  0.9694444 13.3409722
[115]  5.7638889  1.0104167  7.9569444  3.6354167  5.0500000  1.9611111
[121]  2.9437500  2.9583333  3.0486111  7.1076389  2.1027778  3.6368056
[127]  7.9743056 21.0000000  1.9375000  1.7256944  5.5354167  5.9437500
[133]  0.6506944  2.0645833  1.4902778  3.4576389  0.9750000  1.0604167
[139] 15.4208333  2.8180556  1.9583333  4.9909722 12.8222222  1.6743056
[145]  2.9590278  0.9979167  9.8777778  1.8715278  3.5819444  2.9423611
[151]  3.4701389  0.9305556  3.1381944  0.9923611  2.8423611  2.0569444
[157]  1.8451389  2.9451389  3.0673611  1.8666667  3.0895833  6.4861111
[163]  6.2673611  2.0256944  0.7645833  3.6201389  5.0388889  9.1215278
[169]  0.8562500  3.9847222  1.9576389  0.7750000  2.7208333  1.6506944
[175]  4.7673611  4.0201389  5.0076389  3.1145833  4.0951389  5.2395833
[181]  1.0208333  1.1944444  3.7020833  3.7965278  1.6256944  9.7013889
[187]  6.6854167  4.9722222  0.6937500  0.3437500  2.3916667  8.8284722
[193] 13.0861111  8.1395833  0.9784722 21.0000000  0.4895833  1.5833333
[199]  8.1138889  0.9951389

In [51]:
rmse <- RMSE(predictions$pred, testing$UnitLengthStay_trunc)
MAE <- MAE(predictions$pred, testing$UnitLengthStay_trunc)
R2 <- R2(predictions$pred, testing$UnitLengthStay_trunc)
cat("RMSE:", rmse, "\n")
cat("MAE:", MAE, "\n")
cat("R2:", R2, "\n")

RMSE: 0.06773893 
MAE: 0.02725363 
R2: 0.9997469 
